<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=829715b1b7b0587cc300bffd1d882c5dec71d24046f34f38f6a3f60088072dea
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-14 10:21:50
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.45 C
-------------------
Today PnL: -17.08 K (-0.12%)
Current PnL: -20.33 L (-13.32%)
CY Booked + Current PnL: -6.81 L (-4.46%)
-------------------
Total profit:  1.91 L
Total loss:  -22.24 L
-------------------
Total Booked + Current PnL: 20.17 L (16.05%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.32%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 91.21 L (62.86%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.36%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,144.98,84.0,M-SC,1.91,160605.0,-13954.0,13989.0,0.37,-7.99,8.71,0.02,234.0,-1.00,1.11,39.70,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,99.57,48.0,M-SC,3.04,86021.0,-14756.0,14847.0,-0.45,-14.64,17.26,0.09,253.0,-0.99,0.59,13.01,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.43,88680.0,-1998.0,16902.0,-0.03,-2.20,19.06,16.44,63.0,-0.12,0.61,17.53,X40,ATH,PHARMA
35,ICICIGI,2252.93,-15.07,59.0,X-MC,7.53,188883.0,14563.0,20645.0,-0.14,8.35,10.93,20.19,68.0,0.71,1.30,24.73,X40,ATH,INSURANCE
43,ITC,452.00,-40.74,43.0,X-LC,0.90,197923.0,-2215.0,23098.0,-0.23,-1.11,11.67,10.44,5.0,-0.10,1.36,4.40,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,39.40,74.0,H-SC,15.22,287314.0,6191.0,126677.0,5.45,2.20,44.09,47.26,171.0,0.05,1.98,53.93,XY24,NTT,INSURANCE
84,WHIRLPOOL,2270.00,-41.44,40.0,M-SC,9.31,114752.0,-46.0,94085.0,1.63,-0.04,81.99,81.92,236.0,-0.00,0.79,35.85,XR,NTT,DURABLES
83,VOLTAS,1530.00,-4.39,47.0,X-MC,3.79,203085.0,11343.0,26421.0,1.27,5.92,13.01,19.69,78.0,0.43,1.40,13.12,XY25,NTT,AC
82,VBL,671.64,-18.42,47.0,X-LC,6.07,292395.0,-23447.0,136110.0,1.17,-7.42,46.55,35.67,4.0,-0.17,2.02,5.40,X40N,ATH,FMCG
5,ANGELONE,3033.00,59.94,74.0,X-SC,7.21,224721.0,33715.0,23978.0,1.00,17.65,10.67,30.21,97.0,1.41,1.55,42.22,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TRIDENT,48.00,-11.17,44.0,M-SC,1.91,69308.0,-22946.0,49250.0,-2.47,-24.87,71.06,28.51,225.0,-0.47,0.48,18.60,XR,NTT,TEXTILES
41,INFY,2275.00,-20.32,54.0,X-LC,4.20,320757.0,7869.0,163811.0,-2.33,2.51,51.07,54.87,2.0,0.05,2.21,11.01,X40,BTT,IT
61,ROUTE,2227.21,-57.12,39.0,H-SC,23.05,70503.0,-62563.0,154444.0,-2.04,-47.02,219.06,69.05,140.0,-0.41,0.49,3.88,SR,ATH,IT
64,SFL,1287.00,27.66,39.0,M-SC,3.02,168642.0,-93576.0,163397.0,-1.99,-35.69,96.89,26.63,239.0,-0.57,1.16,4.85,XY24,NTT,MISC
59,RELAXO,1176.00,-41.92,36.0,X-SC,3.46,79204.0,-65956.0,141886.0,-1.92,-45.44,179.14,52.31,91.0,-0.46,0.55,6.02,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.44,40.0,M-SC,9.31,114752.0,-46.0,94085.0,1.63,-0.04,81.99,81.92,236.0,-0.00,0.79,35.85,XR,NTT,DURABLES
25,FINCABLES,1641.55,-19.34,45.0,M-SC,10.36,146440.0,-1204.0,160528.0,-0.79,-0.82,109.62,107.91,216.0,-0.01,1.01,2.16,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-14.08,38.0,H-MC,5.75,209188.0,-386.0,75057.0,0.25,-0.18,35.88,35.63,119.0,-0.01,1.44,16.46,AR,ATH,PHARMA
85,WIPRO,420.00,-14.70,50.0,M-LC,5.67,150598.0,-347.0,109801.0,-0.99,-0.23,72.91,72.51,99.0,-0.00,1.04,5.78,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-19.34,45.0,M-SC,10.36,146440.0,-1204.0,160528.0,-0.79,-0.82,109.62,107.91,216.0,-0.01,1.01,2.16,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,144.98,84.0,M-SC,1.91,160605.0,-13954.0,13989.0,0.37,-7.99,8.71,0.02,234.0,-1.00,1.11,39.70,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,99.57,48.0,M-SC,3.04,86021.0,-14756.0,14847.0,-0.45,-14.64,17.26,0.09,253.0,-0.99,0.59,13.01,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.17,47.0,H-SC,2.44,222084.0,-47583.0,83926.0,-1.03,-17.65,37.79,13.47,158.0,-0.57,1.53,12.92,XY24,NTT,PAINTS
67,SIS,528.00,2020.21,47.0,H-SC,2.42,85565.0,-25467.0,49072.0,-0.10,-22.94,57.35,21.26,166.0,-0.52,0.59,15.61,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,39.40,74.0,H-SC,15.22,287314.0,6191.0,126677.0,5.45,2.20,44.09,47.26,171.0,0.05,1.98,53.93,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,18.70,55.0,M-SC,36.52,160200.0,10980.0,133206.0,-0.19,7.36,83.15,96.62,245.0,0.08,1.10,73.37,XY25,NTT,FINANCE
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.62,131106.0,7770.0,123856.0,-0.14,6.30,94.47,106.72,139.0,0.06,0.90,31.15,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.62,131106.0,7770.0,123856.0,-0.14,6.30,94.47,106.72,139.0,0.06,0.90,31.15,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-41.44,40.0,M-SC,9.31,114752.0,-46.0,94085.0,1.63,-0.04,81.99,81.92,236.0,-0.00,0.79,35.85,XR,NTT,DURABLES
85,WIPRO,420.00,-14.70,50.0,M-LC,5.67,150598.0,-347.0,109801.0,-0.99,-0.23,72.91,72.51,99.0,-0.00,1.04,5.78,XR,NTT,IT
86,ZYDUSLIFE,1286.17,-14.08,38.0,H-MC,5.75,209188.0,-386.0,75057.0,0.25,-0.18,35.88,35.63,119.0,-0.01,1.44,16.46,AR,ATH,PHARMA
25,FINCABLES,1641.55,-19.34,45.0,M-SC,10.36,146440.0,-1204.0,160528.0,-0.79,-0.82,109.62,107.91,216.0,-0.01,1.01,2.16,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,12.20,27.0,X-SC,9.35,85598.0,-43072.0,88371.0,-1.44,-33.47,103.24,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.17,28.0,X-SC,37.30,46518.0,-18488.0,169419.0,-1.13,-28.44,364.20,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
26,GILLETTE,11206.78,-17.68,29.0,X-SC,5.89,254235.0,339.0,81965.0,0.13,0.13,32.24,32.42,85.0,0.00,1.75,15.25,X40,ATH,FMCG
27,GLAXO,3466.20,-19.65,29.0,X-MC,5.83,199504.0,220.0,77787.0,-0.03,0.11,38.99,39.15,60.0,0.00,1.38,29.14,X40,ATH,PHARMA
46,JSWINFRA,342.00,-19.06,32.0,X-MC,5.37,189776.0,-10571.0,41068.0,-0.07,-5.28,21.64,15.23,66.0,-0.26,1.31,25.27,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-15.75,41.0,X-LC,0.13,259184.0,-17226.0,438384.0,-0.57,-6.23,169.14,152.37,3.0,-0.04,1.79,1.24,XY24,NTT,AUTO
43,ITC,452.00,-40.74,43.0,X-LC,0.90,197923.0,-2215.0,23098.0,-0.23,-1.11,11.67,10.44,5.0,-0.10,1.36,4.40,X40,NTT,FMCG
34,HONAUT,58357.33,-24.86,38.0,X-SC,0.95,106800.0,-21138.0,68277.0,0.24,-16.52,63.93,36.84,90.0,-0.31,0.74,9.34,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.00,67.0,X-MC,1.01,184766.0,8028.0,46709.0,0.08,4.54,25.28,30.97,75.0,0.17,1.27,17.67,X40,ATH,INSURANCE
66,SIEMENS,4671.50,-5.41,46.0,H-LC,1.64,153985.0,-32110.0,79595.0,0.44,-17.25,51.69,25.51,49.0,-0.40,1.06,13.47,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,12.20,27.0,X-SC,9.35,85598.0,-43072.0,88371.0,-1.44,-33.47,103.24,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.17,28.0,X-SC,37.30,46518.0,-18488.0,169419.0,-1.13,-28.44,364.20,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
53,PAGEIND,51769.93,-28.58,38.0,X-MC,7.67,158840.0,-4900.0,48240.0,0.32,-2.99,30.37,26.47,55.0,-0.10,1.09,0.64,X40,ATH,APPARELS
76,TMPV,1065.00,-15.75,41.0,X-LC,0.13,259184.0,-17226.0,438384.0,-0.57,-6.23,169.14,152.37,3.0,-0.04,1.79,1.24,XY24,NTT,AUTO
10,BAJAJHFL,181.50,-18.37,36.0,X-MC,6.54,176218.0,-24288.0,121449.0,-0.18,-12.11,68.92,48.45,64.0,-0.20,1.21,1.62,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-53.17,28.0,X-SC,37.30,46518.0,-18488.0,169419.0,-1.13,-28.44,364.20,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-41.92,36.0,X-SC,3.46,79204.0,-65956.0,141886.0,-1.92,-45.44,179.14,52.31,91.0,-0.46,0.55,6.02,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,12.20,27.0,X-SC,9.35,85598.0,-43072.0,88371.0,-1.44,-33.47,103.24,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.43,88680.0,-1998.0,16902.0,-0.03,-2.20,19.06,16.44,63.0,-0.12,0.61,17.53,X40,ATH,PHARMA
34,HONAUT,58357.33,-24.86,38.0,X-SC,0.95,106800.0,-21138.0,68277.0,0.24,-16.52,63.93,36.84,90.0,-0.31,0.74,9.34,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.86,57.0,X-LC,8.64,290310.0,-55646.0,122337.0,-0.56,-16.08,42.14,19.28,1.0,-0.45,2.00,7.32,X40,ATH,IT
41,INFY,2275.00,-20.32,54.0,X-LC,4.20,320757.0,7869.0,163811.0,-2.33,2.51,51.07,54.87,2.0,0.05,2.21,11.01,X40,BTT,IT
76,TMPV,1065.00,-15.75,41.0,X-LC,0.13,259184.0,-17226.0,438384.0,-0.57,-6.23,169.14,152.37,3.0,-0.04,1.79,1.24,XY24,NTT,AUTO
82,VBL,671.64,-18.42,47.0,X-LC,6.07,292395.0,-23447.0,136110.0,1.17,-7.42,46.55,35.67,4.0,-0.17,2.02,5.40,X40N,ATH,FMCG
43,ITC,452.00,-40.74,43.0,X-LC,0.90,197923.0,-2215.0,23098.0,-0.23,-1.11,11.67,10.44,5.0,-0.10,1.36,4.40,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-708.79,78.0,L-MC,6.48,49305.0,-27902.0,53294.0,0.18,-36.14,108.09,32.89,259.0,-0.52,0.34,35.78,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,43.0,L-SC,21.79,76800.0,-36749.0,76800.0,-1.06,-32.36,100.00,35.27,270.0,-0.48,0.53,87.97,XR,NTT,HOTELS
7,ASIANTILES,137.00,7422.22,59.0,L-SC,16.32,82539.0,-11271.0,87888.0,-1.07,-12.01,106.48,81.67,271.0,-0.13,0.57,59.84,XR,NTT,CERAMICS
51,MASFIN,398.61,-19.60,48.0,H-SC,7.46,91530.0,-6450.0,28054.0,-0.38,-6.58,30.65,22.05,164.0,-0.23,0.63,33.05,XR,ATH,FINANCE
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.62,131106.0,7770.0,123856.0,-0.14,6.30,94.47,106.72,139.0,0.06,0.90,31.15,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.07,86.0,X-LC,8.81,254417.0,2649.0,50553.0,0.41,1.05,19.87,21.13,31.0,0.05,1.75,36.85,X40,ATH,PAINTS
39,INDIGOPNTS,1408.00,144.98,84.0,M-SC,1.91,160605.0,-13954.0,13989.0,0.37,-7.99,8.71,0.02,234.0,-1.00,1.11,39.70,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,70.25,58.0,H-SC,8.40,147571.0,-35204.0,146272.0,-1.38,-19.26,99.12,60.77,159.0,-0.24,1.02,32.90,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7422.22,59.0,L-SC,16.32,82539.0,-11271.0,87888.0,-1.07,-12.01,106.48,81.67,271.0,-0.13,0.57,59.84,XR,NTT,CERAMICS
40,INDUSINDBK,1800.00,-708.79,78.0,L-MC,6.48,49305.0,-27902.0,53294.0,0.18,-36.14,108.09,32.89,259.0,-0.52,0.34,35.78,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,44.49
2,50,75.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.59
MC    27.70
LC    24.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.49
X40      20.49
X40N     12.28
XY25     10.05
XR        9.53
AR        8.98
OX40N     7.53
X200      1.80
SR        1.01
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.20
X-MC    21.34
X-LC    19.81
M-SC    13.37
X-SC     8.58
H-MC     4.65
H-LC     2.86
L-SC     1.44
M-MC     1.37
M-LC     1.04
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.59,-4.57,38.39
IT,12.47,-16.15,78.25
FINANCE,11.92,-9.28,58.82
MISC,7.56,-18.41,78.13
PAINTS,6.00,-6.13,21.95
ELECTRICAL,5.55,-12.33,52.16
INSURANCE,4.55,4.35,29.36
AUTO,3.47,-14.25,123.79
PHARMA,3.43,-0.44,34.13


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3317905.0,20
XR,1272716.0,13
AR,1268282.0,10
X40,994018.0,14
OX40N,701227.0,10
X40N,689971.0,9
XY25,475706.0,7
SR,274178.0,2
MH,64558.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3491679.0,25
M-SC,1412132.0,16
X-LC,1193296.0,11
X-MC,1146518.0,15
X-SC,831980.0,8
H-MC,403537.0,3
L-SC,253000.0,3
H-LC,142025.0,2
M-LC,109801.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1169324.0      6
           AR         843328.0      5
           XR         801598.0      7
M-SC       XY24       735433.0      6
X-LC       XY24       502149.0      2
X-MC       X40        412113.0      7
X-LC       X40        411569.0      5
X-SC       XY24       375491.0      2
X-MC       XY24       351394.0      3
H-SC       OX40N      338693.0      4
X-SC       X40N       286153.0      4
M-SC       OX40N      274222.0      5
H-SC       SR         274178.0      2
H-MC       AR         219423.0      2
X-LC       X40N       203056.0      2
X-MC       X40N       200762.0      3
H-MC       XY24       184114.0      1
X-MC       XY25       182249.0      2
X-SC       X40        170336.0      2
L-SC       XR         164688.0      2
M-SC       XR         143335.0      2
           XY25       133206.0      1
           AR         125936.0      2
M-LC       XR         109801.0      1
L-SC       OX40N       88312.0      1
H-LC       AR          79595.0      1
X-LC       XY25        76522.0      2
H-SC       MH          64558.0      1
H-LC       X200        62430.0      1
L-MC       XR          53294.0      1
M-MC       XY25        46759.0      1
L-LC       XY25        36970.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
